# Ownership-Networks

## Data Import

In [1]:
import pandas as pd
import numpy as np

In [2]:
codebook = 'new'

In [3]:
file_name = '3030-own'
df = pd.read_csv(f'Data/{file_name}.csv',
                     encoding='unicode_escape',
                    index_col=0)

df.rename(columns={'Company name Latin alphabet': 'Comp_Name',
                   'Country ISO code': 'Country',
                   'NACE Rev. 2, core code (4 digits)': 'NACE',
                   'BvD ID number': 'BvD',
                   'Operating revenue (Turnover)\nth USD Last avail. yr': 'TURN',
                   'Cash flow\nth USD Last avail. yr': 'CF',
                   'Total assets\nth USD Last avail. yr': 'TASS',
                   'Shareholders funds\nth USD Last avail. yr': 'EC',
                   'Number of employees\nLast avail. yr': 'EM',
                   'Shareholder - Name': 'Sha_name',
                   'Shareholder - BvD ID number': 'Sha_BvD',
                   'Shareholder - Direct %': 'Sha_%',
                   'Shareholder - Country ISO code': 'Sha_cntr',
                   'Shareholder - NACE,\nCore code': 'Sha_NACE', 
                   'Subsidiary - Name': 'Sub_name',
                   'Subsidiary - BvD ID number': 'Sub_BvD',
                   'Subsidiary - Direct %': 'Sub_%',
                   'Subsidiary - Country ISO code': 'Sub_cntr',
                   'Subsidiary - NACE,\nCore code': 'Sub_NACE'
                  },
            inplace=True)

In [4]:
df.head()

,Comp_Name,Country,BvD,NACE,TURN,CF,TASS,EC,EM,Sha_name,Sha_BvD,Sha_cntr,Sha_NACE,Sha_%,Sub_name,Sub_BvD,Sub_cntr,Sub_NACE,Sub_%
1.0,AIRBUS SE,NL,NL24288945,3030.0,59 736 738,7 471 764,121 241 465,10 743 847,126 495,GESELLSCHAFT ZUR BETEILIGUNGSVERWALTUNG GZBV M...,DE6070642164,DE,6420,10.90,AERO RÉ,LULB51757,LU,6520.0,100.00
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SOGEPA,FR318186756,FR,6420,10.90,AIRBUS BANK GMBH,DEFEB14827,DE,6419.0,100.00
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAPITAL RESEARCH & MANAGEMENT CO,US149114345L,US,6530,9.63,AIRBUS DEFENCE AND SPACE LIMITED,GB02449259,GB,3299.0,100.00
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SOCIEDAD ESTATAL DE PARTICIPACIONES INDUSTRIALES,ESQ2820015B,ES,8411,4.12,AIRBUS DEFENCE AND SPACE NETHERLANDS B.V.,NL28086907,NL,3030.0,100.00
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCI FUND MANAGEMENT LTD,YY*4000000169624,n.a.,NaN,3.02,AUSTRALIAN AEROSPACE MILITARY MAINTENANCE PTY LTD,AU2-88271,AU,NaN,100.00


## Codebook

In [5]:
# If it has already been generated
if codebook == 'old':   
    cdbk = pd.read_csv(f'Data\\Codebook.csv',index_col=0)
    # Creating dictionary so that it can be used for index in the main dataframe
    di_cdbk = dict([(bvd, index) for index, bvd in zip(cdbk.index, cdbk.BvD)])

elif codebook == 'new':
    # Add Targi companies
    count = 1
    di_cdbk = {}
    df_targi = df[['Comp_Name', 'BvD']].dropna().drop_duplicates(subset=['Comp_Name', 'BvD']).sort_values(by='Comp_Name')
    for company, bvd_code in zip(df_targi['Comp_Name'], df_targi['BvD']):
        index = f'Targi_{count}'
        di_cdbk[index] = [company, bvd_code]
        count += 1
        
    # Add Neigh companies
    count = 1
    # Get all companies 'relative' to Targi's 
    relatives = set(list(df['Sha_BvD'].unique()) + list(df['Sub_BvD'].unique()))
    targi = [value[1] for value in di_cdbk.values()]
    # Extract those that are not already coded as Targi
    neigh = [neigh for neigh in relatives if neigh not in targi]
    neigh.remove(neigh[0])
    df2 = df[['Sha_name', 'Sha_BvD']].rename(columns={'Sha_name':'Comp_Name', 'Sha_BvD':'BvD'}).dropna()
    df3 = df[['Sub_name', 'Sub_BvD']].rename(columns={'Sub_name':'Comp_Name', 'Sub_BvD':'BvD'}).dropna()
    df4 = pd.concat([df2, df3])
    df4 = df4[df4['BvD'].isin(neigh)].drop_duplicates(subset=['Comp_Name', 'BvD']).sort_values(by='Comp_Name')
    
    for company, bvd_code in zip(df4['Comp_Name'], df4['BvD']):
        index = f'Neigh_{count}'
        di_cdbk[index] = [company, bvd_code]
        count += 1
    
    # It is ready for export
    cdbk_all = pd.DataFrame.from_dict(di_cdbk, orient='index', columns=['Comp_Name', 'BvD'])  # orient='index',
#     cdbk_all.to_csv('Data\Codebook.csv')

In [6]:
# For mapping on companies there must a simpler two-column codeboook
di_all = dict([(bvd, index) for index, bvd in zip(cdbk_all.index, cdbk_all.BvD)])

## Creating separate dataframe with economic attributes for Targi companies

In [7]:
import copy
df_econ = copy.copy(df[['Comp_Name', 'Country', 'NACE', 'BvD', 'TURN', 'CF',
       'TASS', 'EC', 'EM']]).drop_duplicates(subset=['BvD'], keep='first').dropna()

df_econ['index'] = df_econ['BvD'].map(di_all)
df_econ.set_index('index', inplace=True)
df_econ.head()

,Comp_Name,Country,NACE,BvD,TURN,CF,TASS,EC,EM
index,,,,,,,,,
Targi_109,AIRBUS SE,NL,3030.0,NL24288945,59 736 738,7 471 764,121 241 465,10 743 847,126 495
Targi_87,AIRBUS,FR,3030.0,FR383474814,46 503 021,1 450 146,67 241 069,11 035 552,8 944
Targi_272,BAE SYSTEMS PLC,GB,3030.0,GB01470151,26 837 099,3 328 945,36 453 156,10 301 190,82 000
Targi_927,SAFRAN,FR,3030.0,FR562082909,18 434 203,1 792 906,47 247 554,15 029 606,77 008
Targi_695,LEONARDO S.P.A.,IT,3030.0,IT00401990585,16 651 490,1 258 319,32 142 064,7 310 935,50 413


In [8]:
df_econ.replace('n.a.', np.nan, inplace=True)

In [9]:
# MISSING DATA in %
df_econ.isna().sum()/len(df_econ)*100

Comp_Name     0.000000
Country       0.000000
NACE          0.000000
BvD           0.000000
TURN         17.062818
CF           27.164686
TASS          5.602716
EC            5.517827
EM           33.955857
dtype: float64

In [10]:
size = ['TURN', 'TASS', 'CF', 'EC', 'EM']

for column in size:
    for value in enumerate(df_econ[column]):
        if type(value[1]) == float:
            df_econ[column].iloc[value[0]] = np.nan
        else:
            df_econ[column].iloc[value[0]] = int(value[1].replace(" ",""))

C:\Users\laptop\AppData\Local\Temp\ipykernel_4652\1455800995.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_econ[column].iloc[value[0]] = int(value[1].replace(" ",""))
C:\Users\laptop\AppData\Local\Temp\ipykernel_4652\1455800995.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_econ[column].iloc[value[0]] = np.nan


In [11]:
# Treats the NA in column with company name and BvD-code in the main dataframe
df['Comp_Name'].fillna(method='ffill', inplace=True)
df['BvD'].fillna(method='ffill', inplace=True)

## Creating separate dataframes for links with shareholders and subsidiaries

In [12]:
import copy
df_sub = copy.copy(df[['Comp_Name', 'BvD','Sub_BvD', 'Sub_%']].dropna(subset='Sub_BvD')).drop('Comp_Name', axis=1)
df_sha = copy.copy(df[['Comp_Name', 'BvD','Sha_BvD', 'Sha_%',]].dropna(subset='Sha_BvD')).drop('Comp_Name', axis=1)

### Subsidiaries Dataframe

In [13]:
df_sub.head()

,BvD,Sub_BvD,Sub_%
1.0,NL24288945,LULB51757,100.00
NaN,NL24288945,DEFEB14827,100.00
NaN,NL24288945,GB02449259,100.00
NaN,NL24288945,NL28086907,100.00
NaN,NL24288945,AU2-88271,100.00


#### Checking for non-numeric values

In [14]:
df_sub[(df_sub['Sub_%'].str.isalpha())]['Sub_%'].value_counts()

WO    54
MO    13
NG     9
VE     3
Name: Sub_%, dtype: int64

#### Changing strings into numeric values

In [15]:
# Converting entire cells
df_sub['Sub_%'].replace({
    'WO': 100,
    'MO': 51,
    'NG': 0.01,
    'VE': 0.01,
    '-': 0
}, 
                        inplace=True)

# Removing special signs from values
import regex
df_sub['Sub_%'] = df_sub['Sub_%'].replace('[<>]+','',regex=True).astype(float)

In [16]:
# Renames columns
df_sub.insert(1, 'source', df_sub['BvD'].map(di_all))
df_sub.insert(2, 'target', df_sub['Sub_BvD'].map(di_all))
df_sub.rename(columns={'Sub_%': 'weight'}, inplace=True)
df_sub.drop(['BvD', 'Sub_BvD'], axis=1, inplace=True)

# Keeps only present links (Drops Nans and weight 0)
df_sub.dropna(inplace=True)

df_sub = df_sub[df_sub['weight'] > 0]
df_sub.head()

,source,target,weight
1.0,Targi_109,Neigh_152,100.0
NaN,Targi_109,Neigh_258,100.0
NaN,Targi_109,Neigh_271,100.0
NaN,Targi_109,Targi_90,100.0
NaN,Targi_109,Neigh_675,100.0


### Shareholders Dataframe

#### Checking for non-numeric values

In [17]:
df_sha[(df_sha['Sha_%'].str.isalpha())]['Sha_%'].value_counts()

WO     40
NG     32
FC     11
MO      9
GP      9
BR      3
T       3
FME     1
Name: Sha_%, dtype: int64

In [18]:
# Converting entire cells
df_sha['Sha_%'].replace({
    'WO': 100,
    'MO': 51,
    'NG': 0.01,
    'VE': 0.01,
    'FC': 0.01,
    'GP': 50,
    'BR': 0.01,
    'T': 0.01,
    'FME': 0.01,
    '-': 0
}, 
                        inplace=True)

# 'WO': Whole ownership,
# 'MO': Major ownership,
# 'NG': Negligable,
# 'VE': ? (Venture Equity?),
# 'FC': ?,
# 'GP': General partnership,
# 'BR': Business Relief,
# 'T': ?,
# 'FME': ? 

In [19]:
# Removing special signs from values
import regex
df_sha['Sha_%'] = df_sha['Sha_%'].replace('[<>]+','',regex=True).astype(float)

In [20]:
# Renames columns
df_sha.insert(1, 'source', df_sha['Sha_BvD'].map(di_all))
df_sha.insert(2, 'target', df_sha['BvD'].map(di_all))
df_sha.rename(columns={'Sha_%': 'weight'}, inplace=True)
df_sha.drop(['BvD', 'Sha_BvD'], axis=1, inplace=True)

# Keeps only present links (Drops Nans and weight 0)
df_sha.dropna(inplace=True)

df_sha = df_sha[df_sha['weight'] > 0]
df_sha.head()

,source,target,weight
1.0,Neigh_1991,Targi_109,10.90
NaN,Neigh_4388,Targi_109,10.90
NaN,Neigh_1287,Targi_109,9.63
NaN,Neigh_4367,Targi_109,4.12
NaN,Neigh_4520,Targi_109,3.02


### All links joined

In [21]:
df_EandN = pd.concat([df_sha, df_sub]).drop_duplicates(subset=['source','target'])

In [22]:
df_EandN.head()

,source,target,weight
1.0,Neigh_1991,Targi_109,10.90
NaN,Neigh_4388,Targi_109,10.90
NaN,Neigh_1287,Targi_109,9.63
NaN,Neigh_4367,Targi_109,4.12
NaN,Neigh_4520,Targi_109,3.02


In [23]:
print(f"Sha links: {len(df_sha)}")
print(f"Sub links: {len(df_sub)}")
print(f"Sum of links: {len(df_sha) + len(df_sub)}")
print(f"All links filtered: {len(df_EandN)}")

Sha links: 1957
Sub links: 1829
Sum of links: 3786
All links filtered: 3626


### % turned into share of EC

### Targi extracted

In [24]:
# Extracting EASIN only links
newDict = { key:value for (key,value) in di_all.items() if str(value).startswith('Targi')}

df_EASIN = df_EandN[(df_EandN['source'].isin(newDict.values())) &
                    (df_EandN['target'].isin(newDict.values()))].reset_index(drop=True)

In [25]:
df_EASIN['weight'] = df_EASIN['weight']/100

## Visualisation

In [26]:
# #Imports
# import networkx as nx
# from pyvis.network import Network

In [27]:
# # Visualisation of PIN
# G1 = nx.from_pandas_edgelist(df_EASIN,
#                            source='source',
#                            target='target',
#                            edge_attr='weight')

# net = Network(notebook=True)
# net.from_nx(G1)
# net.show('example.html')